In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate,PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_structured_chat_agent,create_react_agent
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from langchain.schema.output_parser import StrOutputParser

from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

import os
import fitz

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [2]:
load_dotenv()

True

In [3]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_36410/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [4]:
llm = ChatOpenAI(model="gpt-4o")

In [5]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [6]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    )
]

In [7]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [8]:
prompt_template_normal_chat=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [9]:
agent_normal_chat = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template_normal_chat)

In [10]:
agent_executor_normal_chat = AgentExecutor(agent=agent_normal_chat, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

## ========================= laptops operations =========================== ##

In [11]:
driver_service = Service(ChromeDriverManager().install())
driver_service.start() 

In [12]:
def google_chrome_search(kw: str)-> str:
    import time
    import random
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    #setup chrome options
    options = Options()

    # Disguise the user agent (make Google think it's a normal user)
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

    # Open Chrome in Incognito mode
    options.add_argument("--incognito")

    # Disable Automation Flag
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Maximize Window
    options.add_argument("--start-maximized")

    # # start Chrome 
    # service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=driver_service, options=options)
    driver.get("https://www.google.com")

    # wait page loading
    time.sleep(random.uniform(1, 3))

    # Find the search box
    search_box = driver.find_element(By.NAME, "q")

    # Simulate mouse movement to the search box
    action = ActionChains(driver)
    action.move_to_element(search_box).perform()

    # Enter keywords
    search_box.send_keys(kw)

    time.sleep(random.uniform(1, 2))

    # Press enter to search
    search_box.send_keys(Keys.RETURN)

    return None

In [13]:
def google_chrome_search_tool(kw: str)-> str:
    google_chrome_search(kw=kw)

    return "AgentFinished, you just need to response the webpage is already open."



In [14]:
def create_stickies_note(note_text):
    import subprocess
    
    # erase special symbols
    note_text = note_text.replace('"', '\\"')
    # allow start a new line
    note_text = note_text.replace("\n", '" & return & "')
    
    applescript = f'''
    tell application "Stickies"
        activate
    end tell
    delay 1
    tell application "System Events"
        keystroke "n" using {{command down}} -- create new stick
        delay 1
        keystroke "{note_text}" -- note input
    end tell
    '''
    subprocess.run(["osascript", "-e", applescript])

    return f"AgentFinished, you just need to response the note is done."

In [15]:
def open_file(file_path):
    """
    open file on macOS(PDF、Word、TXT)
    :param file_path: the file path
    """
    import subprocess
    # make sure the path exists.
    if not os.path.exists(file_path):
        print(f"error: file '{file_path}' does not exist.")
        return
    
    try:
        subprocess.run(["open", file_path], check=True)

        return "AgentFinished, you just need to response the file is opened."
    
    except subprocess.CalledProcessError as e:

        return f"cannot open the file: {e}" 

In [16]:
system_message=SystemMessagePromptTemplate.from_template(
        """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a JSON blob to specify a tool by providing an "action" key (tool name) and an "action_input" key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        
        **STRICT FORMAT**:
        - Always respond in JSON format.
        - DO NOT add extra words outside of JSON.
        - **ONLY output JSON**, nothing else.

        Example:
        ```
        {{
            "action": "GoogleChromeSearch",
            "action_input": "current NBA trade news"
        }}
        ```

        **If you have the final answer, respond with:**
        ```
        {{
            "action": "Final Answer",
            "action_input": "Final response to human"
        }}
        ```

        **IMPORTANT: DO NOT ADD ANY EXTRA TEXT OUTSIDE JSON!**
    """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [17]:
prompt_template_laptop_operations=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [18]:
laptop_operations_tools = [
    Tool(
        name="CreateNote",
        func=create_stickies_note,
        description="Use this tool to write down a note"
    ),
    Tool(
        name="GoogleChromeSearch",
        func=google_chrome_search_tool,
        description="Use this tool to search some web pages through google if asking you search something online"
    ),
    Tool(
        name="OpenLocalPath",
        func=open_file,
        description="Use this tool to open the local file."
    )
]

In [19]:
agent_laptop_operations = create_structured_chat_agent(llm=llm, tools=laptop_operations_tools, prompt=prompt_template_laptop_operations)

In [20]:
# agent_laptop_operations = create_react_agent(llm=llm,tools=tools,prompt=prompt_template_laptop_operations,stop_sequence=True,)

In [21]:
agent_executor_laptop_operations = AgentExecutor(agent=agent_laptop_operations, tools=laptop_operations_tools, memory=memory,handle_parsing_errors=True, verbose=True)

## ============================= RAG retriever =========================== ##

In [22]:
fire_dir_with_notes="../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf"
fire_dir_origin="../../../../NOTES/papers/inprogress/Attentionisallyouneed.pdf"

In [23]:
db_dir = os.path.join(os.getcwd(), "db")
persistent_directory = os.path.join(db_dir, "chroma_db")

In [24]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # open pdf file
    text = ""
    for page in doc:  # recursive each page
        text += page.get_text("text") + "\n"  # Extract text and wrap
    return text.strip()

In [25]:
# extract text from pdf
pdf_text=extract_text_from_pdf(fire_dir_origin)

In [26]:
def create_vector_store(text:str,metadata:str,chunk_size:int=1000,chunk_overlap:int=100,embedding_model_name:str="text-embedding-3-small"):
    # combine text content and file metadata together
    documents = [Document(page_content=pdf_text, metadata={"source": metadata})]

    # Splits text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)

    # Define the embedding model
    embeddings = OpenAIEmbeddings(model=embedding_model_name)

    # create chroma db
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory
    )

    return db

In [27]:
db=create_vector_store(text=pdf_text,metadata=fire_dir_with_notes)

Created a chunk of size 2857, which is longer than the specified 1000
Created a chunk of size 4255, which is longer than the specified 1000
Created a chunk of size 1826, which is longer than the specified 1000
Created a chunk of size 2507, which is longer than the specified 1000
Created a chunk of size 3192, which is longer than the specified 1000
Created a chunk of size 3472, which is longer than the specified 1000
Created a chunk of size 3320, which is longer than the specified 1000
Created a chunk of size 3193, which is longer than the specified 1000
Created a chunk of size 2972, which is longer than the specified 1000
Created a chunk of size 3111, which is longer than the specified 1000
Created a chunk of size 3215, which is longer than the specified 1000
Created a chunk of size 3213, which is longer than the specified 1000


In [28]:
# Define the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Load create chroma vecstore with the corresponding embedding function
pdf_db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_36410/2773572095.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  pdf_db = Chroma(persist_directory=persistent_directory,


In [29]:
# create rag retriever
retriever = db.as_retriever(search_type="similarity",search_kwargs={"k": 1})

In [30]:
system_message=SystemMessagePromptTemplate.from_template(
    """ Given the following conversation and the input question, rephrase the input question to be a standalone question.

        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
     
        """    
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    """
)


contextualize_prompt=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message
])

In [31]:
# reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm=llm, retriever=retriever, prompt=contextualize_prompt
)

In [32]:
system_message=SystemMessagePromptTemplate.from_template(
    """
    You are an AI assistant. Use the retrieved context below to answer questions.
    If you don't know the answer, just say so.
    {context}
    
    **Answer each question in 3 short sentences. Make the answer concise and clear**

    Provide only ONE action per $JSON_BLOB, as shown:
    ```
    {{
    "action": $TOOL_NAME,
    "action_input": $INPUT
    }}
    ```
    Follow this format:

    Question: input question to answer
    Thought: consider previous and subsequent steps
    Action:
    ```
    $JSON_BLOB
    ```
    Observation: action result
    ... (repeat Thought/Action/Observation N times)
    Thought: I know what to respond
    Action:
    ```
    {{
    "action": "Final Answer",
    "action_input": "Final response to human"
    }}
    ```        
    Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation 
    """
)

qa_prompt=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message
])

In [33]:
# Create a chain for passing a list of Documents to a model.
# feeds all retrieved context into the LLM

qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [34]:
# First use history_aware_retriever to adjust the problem (complete historical information)
# Then use retriever to query the knowledge base.
# Finally, the query result is passed to question_answer_chain to generate the final answer.
rag_chain = create_retrieval_chain(retriever=history_aware_retriever, combine_docs_chain=qa_chain)

In [35]:
rag_tools = [
    Tool(
        name="Answer Question with local knowledge",
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "history": memory.chat_memory.messages,"agent_scratchpad":""}
        ),
        description="Use RAG to answer questions based on retrieved knowledge on local laptop.",
    )
]

In [36]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
     
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [37]:
prompt_template_rag=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [38]:
agent_rag = create_structured_chat_agent(llm=llm, tools=rag_tools, prompt=prompt_template_rag)

In [39]:
agent_executor_rag = AgentExecutor(agent=agent_rag, tools=rag_tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [40]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible."""
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """
        Classify the question as normal, operations or retriever: {input}.

        normal:
        -Ask the time.
        -Daily greetings.
        -Random Querstions.

        operations:
        -Open some applications.
        -Write some notes.
        -find some files.

        retriever:
        -Conclude information form local knowledge.
        -Information from my laptap.
        -Where the corresponding file path on my laptop.

        You need to ensure that you response the quesntion with 2 parts:
        First part: choose one from these choices: 'normal', 'operations' or 'retriever'.
        Second part: re-write the question with the history memory as a reference in clear sentences.
        Use - to concatenate these 2 parts
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)


prompt_template_classification=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [41]:
# prompt_template_classification = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant."),
#         (
#         "human",
#         """
#         Classify the question as normal or operations: {input}.

#         normal:
#         -Ask the time.
#         -Daily greetings.
#         -Random Querstions.

#         operations:
#         -Open some applications.
#         -Write some notes.
#         -find some files.

#         You need to ensure that you response the quesntion with 2 parts:
#         First part: choose one from these choices: 'normal' or 'operations'.
#         Second part: re-write the question with the history memory as a reference in clear sentences.
#         Use - to concatenate these 2 parts
#          """
#         ),
#         MessagesPlaceholder("history",optional=True)
#     ]
# )

In [42]:
branches = RunnableBranch(
    (
        lambda x: "normal" in x.get("input", "").lower(),
        RunnableLambda(lambda x: agent_executor_normal_chat.invoke(x))
    ),
    (
        lambda x: "operations" in x.get("input", "").lower(),
        RunnableLambda(lambda x: agent_executor_laptop_operations.invoke(x))
    ),
    (
        lambda x: "retriever" in x.get("input", "").lower(),
        RunnableLambda(lambda x: agent_executor_rag.invoke({"input": x["input"], "history": memory.chat_memory.messages}))
    ),
    RunnableLambda(lambda x: {"output": "I didn't understand your request."})
)

In [43]:
classification_chain = prompt_template_classification | llm | StrOutputParser()

In [44]:
# classification_runnable = RunnableLambda(lambda x: {"input": classification_chain.invoke(x)})

In [45]:
# classification_runnable = RunnableLambda(
#     lambda x: {
#         "input": classification_chain.invoke({"input": x["input"], "history": memory.chat_memory.messages}),
#         "history": memory.chat_memory.messages  
#     }
# )


# we cannot record the input question into memory if we do like this. 
# we need to follow the below approach to store all history into memory.

In [46]:
classification_runnable = RunnableLambda(
    lambda x: {
        "history": memory.chat_memory.add_user_message(x["input"]) or memory.chat_memory.messages,
        "input": classification_chain.invoke({"input": x["input"], "history": memory.chat_memory.messages}),
    }
)

In [47]:
chain = classification_runnable | branches

In [48]:
result=chain.invoke({"input":"Check the local knowledge and tell me what is transformer?"})



> Entering new AgentExecutor chain...

Thought: I need to use local knowledge to provide an accurate explanation of what a transformer is.
Action:
```json
{
  "action": "Answer Question with local knowledge",
  "action_input": "What is a transformer?"
}
```{'input': 'What is a transformer?', 'history': [HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={})], 'agent_scratchpad': '', 'context': [Document(metadata={'source': '../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf'}, page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1\nEncoder and Decoder Stacks\nEncoder:\nThe encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The first is

In [49]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={}), HumanMessage(content='```json\n{\n  "response": "retriever-You\'re asking to gather local knowledge about what a transformer is, by accessing information possibly from your laptop or accessible files."\n}\n```', additional_kwargs={}, response_metadata={}), AIMessage(content="A transformer is a neural network architecture that uses stacked self-attention and point-wise fully connected layers for both its encoder and decoder components. It's crafted to work with sequential data to make models more efficient in tasks like language translation. The encoder uses layers with multi-head self-attention mechanisms, and the decoder has extra layers for focusing on the encoder's output while making sure predictions can't use future information.", additional_kwargs={}, response_metadata={})]), retur

In [50]:
result=chain.invoke({"input":"Where is the paper exist on my laptop?"})



> Entering new AgentExecutor chain...
Thought: To find the location of the transformer paper on your laptop, I'll need to use the tool to access local knowledge about files.
Action:
```json
{
  "action": "Answer Question with local knowledge",
  "action_input": "Where is the 'Attention is All You Need' paper stored on my laptop?"
}
```{'input': "Where is the 'Attention is All You Need' paper stored on my laptop?", 'history': [HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={}), HumanMessage(content='```json\n{\n  "response": "retriever-You\'re asking to gather local knowledge about what a transformer is, by accessing information possibly from your laptop or accessible files."\n}\n```', additional_kwargs={}, response_metadata={}), AIMessage(content="A transformer is a neural network architecture that uses stacked self-attention and point-wise fully connected layers for both its encoder and decoder components. I

In [51]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={}), HumanMessage(content='```json\n{\n  "response": "retriever-You\'re asking to gather local knowledge about what a transformer is, by accessing information possibly from your laptop or accessible files."\n}\n```', additional_kwargs={}, response_metadata={}), AIMessage(content="A transformer is a neural network architecture that uses stacked self-attention and point-wise fully connected layers for both its encoder and decoder components. It's crafted to work with sequential data to make models more efficient in tasks like language translation. The encoder uses layers with multi-head self-attention mechanisms, and the decoder has extra layers for focusing on the encoder's output while making sure predictions can't use future information.", additional_kwargs={}, response_metadata={}), HumanMe

In [52]:
result=chain.invoke({"input":"Could you help me to open the web page for the current NBA trade news?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "GoogleChromeSearch",
    "action_input": "current NBA trade news"
}
```AgentFinished, you just need to response the webpage is already open.```json
{
    "action": "Final Answer",
    "action_input": "The webpage for the current NBA trade news is now open. You can check the latest updates there!"
}
```

> Finished chain.


In [53]:
result=chain.invoke({"input":"Help me open the file."})



> Entering new AgentExecutor chain...
```json
{
    "action": "OpenLocalPath",
    "action_input": "../../../../NOTES/papers/inprogress/Attention-is-all-you-need.pdf"
}
```AgentFinished, you just need to response the file is opened.```json
{
    "action": "Final Answer",
    "action_input": "The file 'Attention-is-all-you-need.pdf' is now open on your laptop. You can go through it whenever you're ready!"
}
```

> Finished chain.


In [54]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={}), HumanMessage(content='```json\n{\n  "response": "retriever-You\'re asking to gather local knowledge about what a transformer is, by accessing information possibly from your laptop or accessible files."\n}\n```', additional_kwargs={}, response_metadata={}), AIMessage(content="A transformer is a neural network architecture that uses stacked self-attention and point-wise fully connected layers for both its encoder and decoder components. It's crafted to work with sequential data to make models more efficient in tasks like language translation. The encoder uses layers with multi-head self-attention mechanisms, and the decoder has extra layers for focusing on the encoder's output while making sure predictions can't use future information.", additional_kwargs={}, response_metadata={}), HumanMe

In [55]:
result=chain.invoke({"input":"what is time now?"})



> Entering new AgentExecutor chain...
{
  "action": "LocalTime",
  "action_input": ""
}Current local time is: 01:35:49```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 01:35 AM."
}
```

> Finished chain.


In [56]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Check the local knowledge and tell me what is transformer?', additional_kwargs={}, response_metadata={}), HumanMessage(content='```json\n{\n  "response": "retriever-You\'re asking to gather local knowledge about what a transformer is, by accessing information possibly from your laptop or accessible files."\n}\n```', additional_kwargs={}, response_metadata={}), AIMessage(content="A transformer is a neural network architecture that uses stacked self-attention and point-wise fully connected layers for both its encoder and decoder components. It's crafted to work with sequential data to make models more efficient in tasks like language translation. The encoder uses layers with multi-head self-attention mechanisms, and the decoder has extra layers for focusing on the encoder's output while making sure predictions can't use future information.", additional_kwargs={}, response_metadata={}), HumanMe